<a href="https://colab.research.google.com/github/Seetureddy/RAG_project/blob/main/RAG_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain langchain-community faiss-cpu llama-cpp-python sentence-transformers pypdf


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade bitsandbytes
!pip install --upgrade accelerate transformers
!pip install --upgrade sentence-transformers


In [ ]:
import os
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader, TextLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline

In [ ]:
from transformers import BitsAndBytesConfig
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer


In [ ]:
from huggingface_hub import login

login()  # This will prompt you to enter your HF token


In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,
    temperature=0.1,
    top_p=0.9,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
<ipython-input-12-3f6f6d7ff2b7>:30: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
documents = []

pdf_loader = PyPDFLoader("/content/drive/MyDrive/SurveySR_DD.pdf")  # Replace with your file path
text_loader = TextLoader("/content/drive/MyDrive/book.txt")  # Replace with your file path

documents.extend(pdf_loader.load())
documents.extend(text_loader.load())

In [ ]:
chunk_size = 500
chunk_overlap = 100
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

chunks = []
for doc in documents:
    doc_chunks = text_splitter.split_text(doc.page_content)
    chunks.extend(doc_chunks)

In [ ]:
chunks = []
for doc in documents:
    doc_chunks = text_splitter.split_text(doc.page_content)  # Split text into chunks
    chunks.extend(doc_chunks)
    print(f"\nDocument split into {len(doc_chunks)} chunks")
    print("Example Chunk:\n", doc_chunks[0][:500])  # Show first chunk for reference

print("\nTotal Chunks Created:", len(chunks))


Document split into 4 chunks
Example Chunk:
 C:\Users\Erin.Antognoli\Desktop\SurveySR_2013_14_DD.accdb Thursday, November 16, 2017
Page: 1Table: FOOD_DES
Properties
AlternateBackShade: 100 AlternateBackThemeColorInd -1
AlternateBackTint: 100 BackShade: 100
BackTint: 100 DatasheetForeThemeColorIn -1
DatasheetGridlinesThemeCol -1 DateCreated: 10/13/2015 1:45:52 PM
DefaultView: 2 DisplayViewsOnSharePointSit 1
FilterOnLoad: False GUID: {guid {7EB42AFD-12DD-46C4-
A90A-90637E9EE1FF}}
HideNewField: False LastUpdated: 10/13/2015 1:53:55 PM

Document split into 3 chunks
Example Chunk:
 C:\Users\Erin.Antognoli\Desktop\SurveySR_2013_14_DD.accdb Thursday, November 16, 2017
Page: 2Table: FOOD_DES
TextAlign: General
Shrt_Desc Short Text 60
AggregateType: -1
AllowZeroLength: False
AppendOnly: False
Attributes: Variable Length
CollatingOrder: General
ColumnHidden: False
ColumnOrder: Default
ColumnWidth: 4845
CurrencyLCID: 0
DataUpdatable: False
GUID: {guid {BB4BD2A8-19EC-4EAD-9F0C-9D9E8BB9F54D}}
Ordi

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_texts(chunks, embedding_model)
retriever = vectorstore.as_retriever()

<ipython-input-15-35390ce169a7>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

In [ ]:
def ask_query(query):
    retrieved_docs = retriever.get_relevant_documents(query)  # Get relevant docs

    #  Print Retrieved Documents
    print("\n Retrieved Documents:")
    for doc in retrieved_docs:
        print(doc.page_content[:500])  # Print first 500 chars of each doc

    if retrieved_docs:
        print("\n Retrieval successful. Generating answer from documents...")
        response = qa_chain.invoke({"query": query})
        return response["result"]

    else:
        print("\n No relevant documents found. Using LLM's general knowledge.")

        fallback_prompt = (
            "You are an intelligent AI assistant. If no relevant documents are found, "
            "answer the question using your general knowledge. Be confident and factual. "
            "Do NOT say 'I don’t know'. Provide a reasonable answer.\n\n"
            f"Question: {query}\n\nAnswer:"
        )

        response = llm.invoke(fallback_prompt)  # Call the LLM directly
        return response


In [ ]:
query = "What is the primary topic discussed in the eBook?"
answer = ask_query(query)
print("\nAnswer:", answer)


🔎 Retrieved Documents:
Section 3. Information about the Project Gutenberg Literary Archive Foundation
This eBook is for the use of anyone anywhere in the United States and most
    other parts of the world at no cost and with almost no restrictions
    whatsoever. You may copy it, give it away or re-use it under the terms
    of the Project Gutenberg License included with this eBook or online
    at www.gutenberg.org. If you
    are not located in the United States, you will have to check the laws
    of the country where you are located before using this eBook.
Section 1. General Terms of Use and Redistributing Project Gutenberg™
electronic works
Section 5. General Information About Project Gutenberg™ electronic works

Professor Michael S. Hart was the originator of the Project
Gutenberg™ concept of a library of electronic works that could be
freely shared with anyone. For forty years, he produced and
distributed Project Gutenberg™ eBooks with only a loose network of
volunteer suppor

In [ ]:
query = "Who is the first president of the United States?"
response = llm.invoke(f"You are a knowledgeable AI. Answer this question:\n\n{query}")
print(response)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


You are a knowledgeable AI. Answer this question:

Who is the first president of the United States?

Answer:
The first president of the United States is George Washington. He was inaugurated as the country's first president on April 30, 1789, and served two terms until March 4, 1


In [ ]:
query = "Who is the fprime minister of India?"
response = llm.invoke(f"You are a knowledgeable AI. Answer this question:\n\n{query}")
print(response)


You are a knowledgeable AI. Answer this question:

Who is the fprime minister of India?

Answer:
The current Prime Minister of India is Narendra Modi. He has been in office since May 2014 and was re-elected for a second term in May 2019.


In [ ]:
query = "What is the national bird of Korea?"
response = llm.invoke(f"You are a knowledgeable AI. Answer this question:\n\n{query}")
print(response)


You are a knowledgeable AI. Answer this question:

What is the national bird of Korea?

Answer: The national bird of Korea is the white-cheeked gibbon (Hypsioma curitum).


In [ ]:
query = "What are the colors in indian national flag?"
response = llm.invoke(f"You are a knowledgeable AI. Answer this question:\n\n{query}")
print(response)


You are a knowledgeable AI. Answer this question:

What are the colors in indian national flag?

Answer: The Indian national flag has three horizontal bands of equal width, colored saffron (orange-yellow) at the top, white in the middle, and green at the bottom.


In [ ]:
query = "Under what license is this eBook available?"
answer = ask_query(query)
print("\nAnswer:", answer)



🔎 Retrieved Documents:
This eBook is for the use of anyone anywhere in the United States and most
    other parts of the world at no cost and with almost no restrictions
    whatsoever. You may copy it, give it away or re-use it under the terms
    of the Project Gutenberg License included with this eBook or online
    at www.gutenberg.org. If you
    are not located in the United States, you will have to check the laws
    of the country where you are located before using this eBook.
easy. You may use this eBook for nearly any purpose such as creation
of derivative works, reports, performances and research. Project
Gutenberg eBooks may be modified and printed and given away—you may
do practically ANYTHING in the United States with eBooks not protected
by U.S. copyright law. Redistribution is subject to the trademark
license, especially commercial redistribution.
of the country where you are located before using this eBook.
  
1.E.2. If an individual Project Gutenberg™ electronic work